# Interactive Analytics

In this notebook you will answer 2 basic analytical questions about the data and visualise the result using Python library Matplotlib. We will see one way how Spark is integrated with Python library Pandas which allows you to access also other libraries of the Python ecosystem, for example Matplotlib for visualisation.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window, count, unix_timestamp, when, lit, ceil

import os
import matplotlib.pyplot as plt

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('Interactive Analytics I')
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

answers_input_path = os.path.join(project_path, 'data/answers')

questions_input_path = os.path.join(project_path, 'output/questions-transformed')

# Task I

* Find out how many answers are being produced per week
* Plot the time evolution: on the x axis have date dimmension, on the y axis have number of answers per week

In [ ]:
answersDF = (
    spark
    .read
    .option('path', answers_input_path)
    .load()
)

In [ ]:
answersDF.show(truncate=25, n=5)

#### Group the data

Hint:
* use `groupBy(window)`, where the `window` will be "1 week"
* docs for [window](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.window.html#pyspark.sql.functions.window)
* the output of grouping by `window` will be struct with two subfields `start` and `end`
* use the `start` subfield and change the type to `date` - this will be used in the plot

In [ ]:
groupedDF = (
    answersDF
    .filter(col('user_id').isNotNull())
    .groupBy(
        window('creation_date', "1 week")
    )
    .agg(
        count('*').alias('answers')
    )
    .withColumn('date', col('window.start').cast('date'))
    .orderBy('window')
)

In [ ]:
groupedDF.show(truncate=False, n=5)

In [ ]:
groupedDF.count()

#### Visualise the data:

Hint
* convert the aggregated data to Pandas dataframe using [toPandas()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.toPandas.html#pyspark.sql.DataFrame.toPandas)
* use ploting options of Pandas dataframe
 * [plot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html)

In [ ]:
local_data = groupedDF.toPandas()

In [ ]:
local_data.plot(
    x='date', y='answers', figsize=(12, 6), 
    title='Number of answers per week',
    legend=False,
    xlabel='Date',
    ylabel='Number of answers',
    kind='line'
)
plt.show()

# Task II

* Compute the response time
 * for each question compute the time it took to have accepted answer
 * consider only questions with accepted answer
* Plot number of answered questions as a function of response time
 * choose hour as the time unit
 * create a bar chart (too see how many questions were answered within first hour, within second hour and so on)
 * plot a cumulative sum (too see for example how many questions in total were answered within first 10 hours and so on)

In [ ]:
questionsDF = (
    spark
    .read
    .option('path', questions_input_path)
    .load()
)

#### Compute response time:

For each question compute how long it took to get accepted answer. Consider only questions that actually have accepted answers.

Hint:
* for each question and answer we now the time when it was created (`created_date`)
* [join](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.join.html#pyspark.sql.DataFrame.join) questions with answers (use `accepted_answer_id` field in the join)
* use [unix_timestamp](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.unix_timestamp.html#pyspark.sql.functions.unix_timestamp) to compare the times
* convert to hours
* [ceil](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.ceil.html#pyspark.sql.functions.ceil) the numbers

In [ ]:
hourly_data = (
    questionsDF.alias('questions')
    .join(answersDF.alias('answers'), questionsDF['accepted_answer_id'] == answersDF['answer_id'])
    .select(
        col('questions.creation_date').alias('question_time'),
        col('answers.creation_date').alias('answer_time')
    )
    .withColumn('response_time', unix_timestamp('answer_time') - unix_timestamp('question_time'))
    .filter(col('response_time') > 0)
    .withColumn('hours', ceil(col('response_time') / 3600))
)

#### Aggregate the data and visualise:

Hint:
* group by hour
* count
* convert to Pandas
* visualize (take first 24 hours)

In [ ]:
hourly_data_grouped = (
    hourly_data
    .groupBy('hours')
    .agg(count('*').alias('cnt'))
    .orderBy('hours')
)

In [ ]:
hourly_data_local = hourly_data_grouped.limit(24).toPandas()

In [ ]:
# inspect the data localy:

hourly_data_local.head(5)

For bar chart you can use df.plot.bar

In [ ]:
hourly_data_local.plot(
    x='hours', y='cnt', figsize=(12, 6), 
    title='Response time of questions',
    legend=False,
    kind='bar',
    xlabel='Hour',
    ylabel='Number of answered questions'
)
plt.show()

#### Note

As you can see, big portion of the questions that have accepted answer were answered within the first hour.

#### Cumulative sum

* To compute cumulative sum you can use [cumsum()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cumsum.html)
* add new col to the Pandas DataFrame as df['new_col'] = df['cnt'].cumsum()

In [ ]:

hourly_data_local['cumsum'] = hourly_data_local['cnt'].cumsum()

hourly_data_local.plot(
    x='hours',
    y='cumsum',
    figsize=(12, 6),
    title='Cumulative size of answered questions',
    xlabel='Hour',
    ylabel='Number of answered questions',
    legend=False
)
plt.show()

In [ ]:
# Also see what is the total number of questions with accepted answer:

(
    questionsDF.alias('questions')
    .join(answersDF.alias('answers'), questionsDF['accepted_answer_id'] == answersDF['answer_id'])
).count()

In [ ]:
spark.stop()